# Customer segmentation

#### Best Customers	111	
Customers who bought most recently, most often and spend the most. Strategy: No price incentives, New products and loyalty programs

#### Loyal Customers	X1X	
Customers who bought most recently. Strategy: Use R and M to further segment.

#### Big Spenders XX1 
Customers who spend the most. Strategy: Market your most expensive products.

#### Almost Lost	311	
Haven't purchased for some time, but purchased frequently and spend the most. Strategy: Agressive price incentives

#### Lost Customers 411	
Haven't purchased for some time, but purchased frequently and spend the most. Strategy: Agressive price incentives.

#### Lost Cheap Customers 444	
Last purchase long ago, purchased few and spend little. Strategy: Don't spend too much trying to re-acquire.

In [154]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [155]:
loans = pd.read_csv('withdrawals-2021_06_01-to-2023_08_01.csv',sep=',')

In [156]:
loans.head()

,withdrawal hash id,withdrawal_id,loan_id,user_id,processor_id,details,amount,requested_currency,rate,raw,pending,processing,completed,status,action,rejected
0,aa13fda4,1344,2175,3174,1,[object Object],14.72,NGN,686.0,[object Object],31/07/2022 13:51,31/07/2022 13:51,2022-07-31,completed,True,NaN
1,2b0ebe24,1343,2174,3363,1,[object Object],109.70,NGN,686.0,[object Object],31/07/2022 13:47,31/07/2022 13:47,2022-07-31,completed,True,NaN
2,1c0e147e,1342,2173,556,1,[object Object],50.00,NGN,686.0,[object Object],30/07/2022 22:03,30/07/2022 22:03,2022-07-30,completed,True,NaN
3,866cf6ce,1341,2170,2663,1,[object Object],150.00,NGN,686.0,[object Object],30/07/2022 0:31,30/07/2022 0:31,2022-07-30,completed,True,NaN
4,9e0bc0a9,1340,2169,256,1,[object Object],150.00,NGN,686.0,[object Object],29/07/2022 10:03,29/07/2022 10:03,2022-07-29,completed,True,NaN


# Create the RFM Table

Since recency is calculated for a point in time and the loan dataset last order date is July 31st 2022, that is the date we will use to calculate recency.

Set this date to the current day and extract all orders until yesterday.

In [157]:
import datetime as dt
NOW = dt.datetime(2022,7,31)

In [158]:
# Make the date_placed column datetime
loans['completed'] = pd.to_datetime(loans['completed'])

Create the RFM table

In [159]:
rfmTable = loans.groupby('user_id').agg({'completed': lambda x: ((NOW - x.max()).days)+1, # Recency
                                        'loan_id': lambda x: len(x),      # Frequency
                                        'amount': lambda x: x.sum()}) # Monetary Value

rfmTable['completed'] = rfmTable['completed'].astype(int)
rfmTable.rename(columns={'completed': 'recency', 
                         'loan_id': 'frequency', 
                         'amount': 'monetary_value'}, inplace=True)

# Validating the RFM Table

In [160]:
rfmTable.head()

,recency,frequency,monetary_value
user_id,,,
1,13,2,12500.00
4,61,1,1078.00
5,294,31,28353.12
7,363,2,8017.00
10,222,4,1500.00


# Determining RFM quantiles

In [161]:
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])
quantiles

,recency,frequency,monetary_value
0.25,67.00,1.0,60.00
0.50,139.00,2.0,300.00
0.75,240.25,4.0,1466.25


Here the three quantiles for our rfm table have been determined. They will be sent to dictionary for easier use below

In [162]:
quantiles = quantiles.to_dict()
quantiles

{'recency': {0.25: 67.0, 0.5: 139.0, 0.75: 240.25},
 'frequency': {0.25: 1.0, 0.5: 2.0, 0.75: 4.0},
 'monetary_value': {0.25: 60.0, 0.5: 300.0, 0.75: 1466.25}}

# Creating RFM segmentation table

In [163]:
rfmSegmentation = rfmTable

We create two classes for the RFM segmentation since, having a high recency is bad, while high frequency and monetary value is good.

In [164]:
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def RClass(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def FMClass(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

In [165]:
rfmSegmentation['R_Quartile'] = rfmSegmentation['recency'].apply(RClass, args=('recency',quantiles,))
rfmSegmentation['F_Quartile'] = rfmSegmentation['frequency'].apply(FMClass, args=('frequency',quantiles,))
rfmSegmentation['M_Quartile'] = rfmSegmentation['monetary_value'].apply(FMClass, args=('monetary_value',quantiles,))

In [166]:
rfmSegmentation['RFMClass'] = rfmSegmentation.R_Quartile.map(str) \
                            + rfmSegmentation.F_Quartile.map(str) \
                            + rfmSegmentation.M_Quartile.map(str)

In [167]:
rfmSegmentation.head()

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
user_id,,,,,,,
1,13,2,12500.00,1,3,1,131
4,61,1,1078.00,1,4,2,142
5,294,31,28353.12,4,1,1,411
7,363,2,8017.00,4,3,1,431
10,222,4,1500.00,3,2,1,321


Who are the top 5 best customers? by RFM Class (111), high spenders who take loans recently and frequently?

In [171]:
rfmSegmentation[rfmSegmentation['RFMClass']=='111'].sort_values('monetary_value', ascending=False).head()

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
user_id,,,,,,,
1790,47,7,13000.00,1,1,1,111
669,10,28,7056.25,1,1,1,111
1311,6,8,4195.69,1,1,1,111
289,9,16,3946.00,1,1,1,111
867,8,12,3605.30,1,1,1,111


In [150]:
rfmSegmentation.to_csv('data.csv')